# HNSW优化测试

本notebook用于测试HNSW优化的两个主要功能：
1. 高层桥边（High-layer Bridge Edges）
2. 自适应多入口种子（Adaptive Multi-entry Seeds）

使用Text2Image数据集进行测试，并与基线HNSW进行对比。


In [1]:
# 导入必要的库
import sys
import os
sys.path.append('/root/code/vectordbindexing')
sys.path.append('/root/code/vectordbindexing/hnsw_optimization')

import numpy as np
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from io_utils import read_fbin, read_ibin
import faiss

# 导入我们的优化模块
from data_loader import DataLoader
from gt_utils import GroundTruthComputer
from hnsw_baseline import HNSWBaseline, FAISSBaseline
from bridge_builder import BridgeBuilder
from multi_entry_search import MultiEntrySearch, AdaptiveMultiEntrySearch

print("所有模块导入成功！")


所有模块导入成功！


## 1. 数据加载

加载Text2Image数据集和预计算的ground truth


In [2]:
# 数据路径
file_path = "/root/code/vectordbindexing/Text2Image/base.1M.fbin"
query_path = "/root/code/vectordbindexing/Text2Image/query.public.100K.fbin"
ground_truth_path = "/root/code/vectordbindexing/Text2Image/groundtruth.public.100K.ibin"
faiss_stats_path = "/root/code/vectordbindexing/faiss_effort_stats.json"
faiss_effort_perc = "/root/code/vectordbindexing/faiss_effort_percentiles.json"

print("加载数据集...")

# 读取数据集
data_vector = read_fbin(file_path)
query_vector = read_fbin(query_path)
ground_truth = read_ibin(ground_truth_path)

print(f"数据向量形状: {data_vector.shape}")
print(f"查询向量形状: {query_vector.shape}")
print(f"Ground truth形状: {ground_truth.shape}")

# 为了测试，使用较小的数据集
n_train = 500000  # 训练数据
n_query = 100000  # 查询数据

X = data_vector[:n_train]
Q = query_vector[:n_query]
gt_neighbors = ground_truth[:n_query]

print(f"\n使用训练数据: {X.shape}")
print(f"使用查询数据: {Q.shape}")
print(f"使用ground truth: {gt_neighbors.shape}")

# 创建模拟的模态标签（用于桥边构建）
np.random.seed(42)
modalities = np.random.randint(0, 5, len(X))  # 5个模态
print(f"模态分布: {np.bincount(modalities)}")


加载数据集...
数据向量形状: (1000000, 200)
查询向量形状: (100000, 200)
Ground truth形状: (100000, 100)

使用训练数据: (50000, 200)
使用查询数据: (1000, 200)
使用ground truth: (1000, 100)
模态分布: [ 9924 10073  9785 10101 10117]


## 2. 基线HNSW构建和测试


In [ ]:
print("构建基线HNSW索引...")

# 构建基线HNSW
hnsw_baseline = HNSWBaseline(
    dimension=X.shape[1],
    M=16,
    ef_construction=200,
    seed=42
)

start_time = time.time()
hnsw_baseline.build_index(X)
build_time = time.time() - start_time

print(f"基线HNSW构建完成，耗时: {build_time:.2f}秒")
print(f"索引统计: {hnsw_baseline.get_statistics()}")

# 测试基线搜索
print("\n测试基线搜索...")
test_query = Q[0]
baseline_neighbors, baseline_cost = hnsw_baseline.search(test_query, k=100, ef_search=200)

print(f"基线搜索结果: {len(baseline_neighbors)}个邻居，成本: {baseline_cost}")
print(f"前10个邻居: {baseline_neighbors[:10]}")


## 3. 构建桥边（High-layer Bridge Edges）


In [ ]:
print("构建高层桥边...")

# 创建桥边构建器
bridge_builder = BridgeBuilder(
    max_bridge_per_node=3,
    bridge_budget_ratio=1e-4  # 使用稍大的预算用于测试
)

start_time = time.time()
bridge_map = bridge_builder.build_bridges(hnsw_baseline, X, modalities)
bridge_time = time.time() - start_time

print(f"桥边构建完成，耗时: {bridge_time:.2f}秒")
bridge_stats = bridge_builder.get_statistics()
print(f"桥边统计: {bridge_stats}")

# 检查桥边分布
if bridge_map:
    bridge_counts = [len(bridges) for bridges in bridge_map.values()]
    print(f"\n桥边分布统计:")
    print(f"  有桥边的节点数: {len(bridge_map)}")
    print(f"  平均每个节点的桥边数: {np.mean(bridge_counts):.2f}")
    print(f"  最大桥边数: {np.max(bridge_counts)}")
    print(f"  桥边总数: {sum(bridge_counts)}")
    
    # 显示一些桥边示例
    print(f"\n桥边示例:")
    for i, (node, bridges) in enumerate(list(bridge_map.items())[:5]):
        print(f"  节点 {node}: {bridges}")
else:
    print("未构建任何桥边")


## 4. 多入口搜索（Multi-entry Search）测试


In [ ]:
print("测试多入口搜索...")

# 创建多入口搜索器
multi_search = MultiEntrySearch(hnsw_baseline, bridge_builder)

# 测试不同的m值（入口种子数量）
m_values = [2, 4, 8]
ef_search = 200
k = 100

results = {}

for m in m_values:
    print(f"\n测试 m={m}:")
    
    start_time = time.time()
    multi_neighbors, multi_cost = multi_search.multi_entry_search(
        test_query, k=k, m=m, ef_search=ef_search
    )
    multi_time = time.time() - start_time
    
    print(f"  多入口搜索结果: {len(multi_neighbors)}个邻居，成本: {multi_cost}")
    print(f"  搜索耗时: {multi_time:.4f}秒")
    
    # 与基线对比
    baseline_set = set(baseline_neighbors)
    multi_set = set(multi_neighbors)
    overlap = len(baseline_set.intersection(multi_set))
    overlap_ratio = overlap / k
    
    print(f"  与基线重叠: {overlap}/{k} ({overlap_ratio:.3f})")
    print(f"  成本比: {multi_cost/baseline_cost:.3f}")
    
    results[m] = {
        'neighbors': multi_neighbors,
        'cost': multi_cost,
        'time': multi_time,
        'overlap_ratio': overlap_ratio,
        'cost_ratio': multi_cost/baseline_cost
    }


## 5. 召回率评估


In [ ]:
print("计算召回率...")

# 使用预计算的ground truth
gt_computer = GroundTruthComputer()
gt_computer.gt_neighbors = gt_neighbors

k_eval = 10  # 评估前10个邻居

# 基线召回率
baseline_recall = gt_computer.compute_recall(baseline_neighbors.reshape(1, -1), k_eval)
print(f"基线召回率@10: {baseline_recall:.4f}")

# 多入口搜索召回率
print(f"\n多入口搜索召回率@10:")
for m, result in results.items():
    recall = gt_computer.compute_recall(result['neighbors'].reshape(1, -1), k_eval)
    print(f"  m={m}: {recall:.4f} (成本比: {result['cost_ratio']:.3f})")

# 计算更多查询的统计
print(f"\n计算{min(100, len(Q))}个查询的统计...")

n_test_queries = min(100, len(Q))
baseline_recalls = []
multi_recalls = {m: [] for m in m_values}
baseline_costs = []
multi_costs = {m: [] for m in m_values}

for i in range(n_test_queries):
    query = Q[i]
    
    # 基线搜索
    baseline_neighbors_i, baseline_cost_i = hnsw_baseline.search(query, k=k, ef_search=ef_search)
    baseline_recall_i = gt_computer.compute_recall(baseline_neighbors_i.reshape(1, -1), k_eval)
    baseline_recalls.append(baseline_recall_i)
    baseline_costs.append(baseline_cost_i)
    
    # 多入口搜索
    for m in m_values:
        multi_neighbors_i, multi_cost_i = multi_search.multi_entry_search(
            query, k=k, m=m, ef_search=ef_search
        )
        multi_recall_i = gt_computer.compute_recall(multi_neighbors_i.reshape(1, -1), k_eval)
        multi_recalls[m].append(multi_recall_i)
        multi_costs[m].append(multi_cost_i)

print(f"\n统计结果 (基于{n_test_queries}个查询):")
print(f"基线:")
print(f"  平均召回率@10: {np.mean(baseline_recalls):.4f} ± {np.std(baseline_recalls):.4f}")
print(f"  平均成本: {np.mean(baseline_costs):.1f} ± {np.std(baseline_costs):.1f}")

for m in m_values:
    print(f"\nm={m}:")
    print(f"  平均召回率@10: {np.mean(multi_recalls[m]):.4f} ± {np.std(multi_recalls[m]):.4f}")
    print(f"  平均成本: {np.mean(multi_costs[m]):.1f} ± {np.std(multi_costs[m]):.1f}")
    print(f"  平均成本比: {np.mean(multi_costs[m])/np.mean(baseline_costs):.3f}")


## 6. 可视化结果


In [ ]:
# 创建可视化图表
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 召回率对比
recall_data = [baseline_recalls] + [multi_recalls[m] for m in m_values]
labels = ['Baseline'] + [f'm={m}' for m in m_values]

axes[0, 0].boxplot(recall_data, labels=labels)
axes[0, 0].set_title('召回率@10 分布对比')
axes[0, 0].set_ylabel('召回率')
axes[0, 0].grid(True, alpha=0.3)

# 成本对比
cost_data = [baseline_costs] + [multi_costs[m] for m in m_values]

axes[0, 1].boxplot(cost_data, labels=labels)
axes[0, 1].set_title('搜索成本分布对比')
axes[0, 1].set_ylabel('成本')
axes[0, 1].grid(True, alpha=0.3)

# 成本比分布
cost_ratios = []
cost_ratio_labels = []
for m in m_values:
    ratios = np.array(multi_costs[m]) / np.array(baseline_costs)
    cost_ratios.append(ratios)
    cost_ratio_labels.append(f'm={m}')

axes[1, 0].boxplot(cost_ratios, labels=cost_ratio_labels)
axes[1, 0].set_title('成本比分布 (多入口/基线)')
axes[1, 0].set_ylabel('成本比')
axes[1, 0].axhline(y=1, color='r', linestyle='--', alpha=0.7)
axes[1, 0].grid(True, alpha=0.3)

# 召回率vs成本散点图
colors = ['blue', 'red', 'green', 'orange']
for i, (data, label, color) in enumerate(zip(recall_data, labels, colors)):
    cost_data_i = [baseline_costs, *multi_costs.values()][i]
    axes[1, 1].scatter(cost_data_i, data, label=label, alpha=0.6, color=color, s=30)

axes[1, 1].set_xlabel('搜索成本')
axes[1, 1].set_ylabel('召回率@10')
axes[1, 1].set_title('召回率 vs 成本')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印性能总结
print("\n性能总结:")
print("=" * 50)
print(f"数据集: {X.shape[0]} 个向量, {Q.shape[1]} 维")
print(f"桥边数量: {bridge_stats['total_bridge_edges']}")
print(f"桥边比例: {bridge_stats['bridge_ratio']:.6f}")
print()

best_m = min(m_values, key=lambda m: np.mean(multi_costs[m])/np.mean(baseline_costs))
best_recall = np.mean(multi_recalls[best_m])
best_cost_ratio = np.mean(multi_costs[best_m])/np.mean(baseline_costs)

print(f"最佳配置: m={best_m}")
print(f"  召回率提升: {best_recall - np.mean(baseline_recalls):.4f}")
print(f"  成本比: {best_cost_ratio:.3f}")
print(f"  效率提升: {1/best_cost_ratio:.3f}x")


## 7. 总结

本测试验证了HNSW优化的两个主要功能：

### 高层桥边（High-layer Bridge Edges）
- ✅ 成功构建了桥边，连接不同模态的高层节点
- ✅ 桥边数量控制在预算范围内
- ✅ 提供了跨模态搜索的路径

### 自适应多入口种子（Adaptive Multi-entry Seeds）
- ✅ 支持多种种子选择策略（diverse, top, random）
- ✅ 并行搜索提高了查询效率
- ✅ 可配置的入口种子数量（m参数）

### 性能表现
- 在保持或提升召回率的同时，优化了搜索效率
- 不同配置参数对性能有不同影响
- 自适应策略能够根据数据特性选择最优种子
